### Code for defining a multiple Gif with matplotlib.

#### We make use of an auxiliar class in order to make the code more easy to read an extend. Any proposition an opinion is welcome!

###### Performance is an issue. for a Set of 300 images, it took about 240 s to make the gif!


In [1]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

from data import cdnet

##### Auxiliary class that encodes some properties of the dataset, such as the name and desired position on the resulting grid

In [2]:
class Batch:
    
    def __init__(self, images, title, position):
        self.images = images
        self.name = title
        self.pos = position

In [3]:
def make_gif(BatchList, Size):
    
    """
    Function make_gif. Recieves a list of Batch objects containing the image data
    and the total size of the grid. Returns a list of matplot images, in order to make 
    the gif. Hence synchronization might not be an issue no more.
    Params:
    batch_List
    
    """
    
    if(not len(BatchList)):
        print("no batches passed!")
        return
    
    if(not type(BatchList[0]) == Batch):
        print("must pass a list of batch")
        
    n_images = len(BatchList[0].images)
    
    if (not len(Size) == 2 or not type(Size[0]) == int or not type(Size[1]) == int):
        print("incorrect input size")
        return
    
    for batch in BatchList:
        if(n_images != len(batch.images)):
            
            print("each batch must have the same number of images")
            return
        
        if (not len(batch.pos) == 2 or not type(batch.pos[0]) == int 
            or not type(batch.pos[1]) == int or batch.pos[0] < 0 or batch.pos[1] < 0):

            print("Image position for batch " + batch.title + 
                  " must be a tuple of integers")
            return
        
        if(batch.pos[0]**Size[1] + batch.pos[1] > Size[0]*Size[1]):
            print("batch " + batch.title + " position out of bounds")
            return
        
        #Define the position of the images in the flattened vector
        batch.pos = batch.pos[0]*Size[1] + batch.pos[1]
        
    #### All comprovations have been done! More could be added

    #Define the grid for the plottings 
    f, axarr = plt.subplots(Size[0], Size[1])
    
    ## shutdown axes
    axes = axarr.flatten()
    [ax.axis('off') for ax in axes]
    
    #get Size of the resulting image
    width, height = f.get_size_inches() * f.get_dpi()
    width, height= int(width), int(height)

    #define the output array
    image_list = np.zeros((len(BatchList[0].images),height, width, 3), dtype='uint8')
    
    canvas = FigureCanvas(f)
    
    #Maybe this could be speeded up
    for i in range(0, len(BatchList[0].images)):
        
        for batch in BatchList:
        
            axes[batch.pos].imshow(batch.images[i])
            axes[batch.pos].set_title(batch.name)
        
        
        canvas.draw() 
        image_list[i] = np.fromstring(canvas.tostring_rgb(), dtype='uint8').reshape(height, width, 3)
    
    return image_list

#### Example:

First, we get the data

In [15]:
ims, gts = cdnet.read_dataset('highway', 1200, 1350,
                              colorspace='rgb', annotated=True)

ims1, gts = cdnet.read_dataset('traffic', 1200, 1350,
                              colorspace='rgb', annotated=True)

ims2, gts = cdnet.read_dataset('fall', 1200, 1350,
                              colorspace='rgb', annotated=True)

Then, we construct the batch objects: 
We assume that the resulting grid will be 2x2

In [18]:
BatchHighway = Batch(ims, "highway", (1,0))
BatchTraffic = Batch(ims1, "traffic", (1,1))
BatchFall= Batch(ims2, "fall", (0,1))

We now call the method as follows:

In [19]:
GigImages = make_gif([BatchHighway, BatchFall, BatchTraffic], (2,2))

And Save the result:

In [22]:
imageio.mimsave("test1.gif", GigImages)

Another configuration:

In [23]:
BatchHighway = Batch(ims, "highway", (0,0))
BatchTraffic = Batch(ims1, "traffic", (1,1))
BatchFall= Batch(ims2, "fall", (2,0))
GigImages = make_gif([BatchHighway, BatchFall, BatchTraffic], (3,2))
imageio.mimsave("test2.gif", GigImages)

Maybe the size of the figure could be a parameter!